In [1]:
import requests
import os # yes or no
from bs4 import BeautifulSoup

def download_files_from_hpa(url, max_size_gb=1, subfolder="downloaded_hpa_files"):
    # Create the subfolder if it doesn't exist
    if not os.path.exists(subfolder):
        os.makedirs(subfolder)

    # Convert the max size from GB to bytes
    max_size_bytes = max_size_gb * 1e9

    # Make an HTTP GET request to the provided URL
    response = requests.get(url)
    response.raise_for_status()  # Ensure we got a successful response

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Search for all <a> tags with the specified href structure
    links = soup.find_all('a', href=True)

    # Base URL to prepend to relative file paths
    base_url = "https://www.proteinatlas.org"

    for link in links:
        file_url = link['href']
        if file_url.endswith('.zip'):  # Check if the link is to a .zip file
            full_url = base_url + file_url

            # Extract filename from the URL
            filename = file_url.split('/')[-1]

            # Create the full path to save the file
            save_path = os.path.join(subfolder, filename)
            
            # Check if the file already exists
            if os.path.exists(save_path):
                print(f"{filename} already exists. Skipping download.")
                continue

            if filename == "proteinatlas.tsv.zip":
                # Check file size without downloading the entire file
                file_response = requests.head(full_url)
                file_size = int(file_response.headers.get('Content-Length', 0))

                if file_size <= max_size_bytes:
                    # Download the file if it's within the size limit
                    print(f"Downloading {filename}...")
                    file_response = requests.get(full_url, stream=True)
                    with open(save_path, 'wb') as file:
                        for chunk in file_response.iter_content(chunk_size=8192):
                            file.write(chunk)
                    print(f"{filename} downloaded!")
                else:
                    print(f"Skipping {filename} as it exceeds the size limit.")

# Example usage
download_files_from_hpa("https://www.proteinatlas.org/about/download")


proteinatlas.tsv.zip already exists. Skipping download.


In [2]:
import zipfile

# Function to unzip a file
def unzip_file(zip_file_path, output_folder_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_folder_path)
        print(f"Unzipped files to {output_folder_path}")

# Unzip the file
unzip_file('downloaded_hpa_files/proteinatlas.tsv.zip', 'unzipped_folder')

# Read the first 5 lines of the unzipped .tsv file
try:
    with open('unzipped_folder/proteinatlas.tsv', 'r') as f:
        for i, line in enumerate(f):
            if i >= 5:
                break
            print(line.strip().replace('\t', ','))
except FileNotFoundError:
    print("The file 'proteinatlas.tsv' was not found in the 'unzipped_folder'.")


Unzipped files to unzipped_folder
Gene,"Gene synonym",Ensembl,"Gene description",Uniprot,Chromosome,Position,"Protein class","Biological process","Molecular function","Disease involvement",Evidence,"HPA evidence","UniProt evidence","NeXtProt evidence","RNA tissue specificity","RNA tissue distribution","RNA tissue specificity score","RNA tissue specific nTPM","RNA single cell type specificity","RNA single cell type distribution","RNA single cell type specificity score","RNA single cell type specific nTPM","RNA cancer specificity","RNA cancer distribution","RNA cancer specificity score","RNA cancer specific FPKM","RNA brain regional specificity","RNA brain regional distribution","RNA brain regional specificity score","RNA brain regional specific nTPM","RNA blood cell specificity","RNA blood cell distribution","RNA blood cell specificity score","RNA blood cell specific nTPM","RNA blood lineage specificity","RNA blood lineage distribution","RNA blood lineage specificity score","RNA blood l

In [3]:
import requests
import os

def download_file_from_github(url, save_path, folder_name="ontology"):
    # Create folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Define the full path to save the file
    full_save_path = os.path.join(folder_name, save_path)

    # Download the file
    response = requests.get(url)
    with open(full_save_path, 'wb') as file:
        file.write(response.content)

    print(f"File downloaded and saved as {full_save_path}")

# URL to the hECA marker gene annotation file
heca_url = "https://github.com/XuegongLab/hECA/raw/main/UHAF/uHAF%20marker%20reference.xlsx"

# Name of the file to save
heca_save_path = "uHAF_marker_reference.xlsx"

# Download the file
download_file_from_github(heca_url, heca_save_path)


File downloaded and saved as ontology/uHAF_marker_reference.xlsx


In [7]:
def convert_to_json(file_path, output_directory, is_hpa):
    # Create dictionaries for tissues/organs and cell types to markers
    tissue_to_marker = defaultdict(set)
    cell_type_to_marker = defaultdict(set)

    if is_hpa:
        # Read the HPA Excel file
        df = pd.read_excel(file_path)

        # Iterate through the DataFrame
        for index, row in df.iterrows():
            tissue = row['Tissue']  # 'Tissue' column used for HPA
            cell_type = row['Cell type']
            marker = row['Marker']
            
            tissue_to_marker[tissue.strip()].add(marker.strip())
            cell_type_to_marker[cell_type.strip()].add(marker.strip())
    else:
        # Read the hECA Excel file
        xls = pd.ExcelFile(file_path)

        # Iterate over each sheet
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name)

            # Check for 'Marker' or 'marker' column
            marker_column = 'Marker' if 'Marker' in df.columns else 'marker'
            if marker_column not in df.columns:
                raise KeyError(f"Column 'Marker' or 'marker' not found in sheet {sheet_name}")

            # Iterate through the DataFrame
            for index, row in df.iterrows():
                tissue = sheet_name  # Use the sheet name as the tissue/organ for hECA
                cell_type = row['cell_type']
                if pd.isnull(cell_type):
                    continue  # Skip rows where cell type is NaN or None
            cell_type = str(cell_type).strip()  # Convert to string and strip whitespace
        
            markers = set(map(str.strip, str(row[marker_column]).split(',')))  # Convert to string and split

            tissue_to_marker[tissue.strip()].update(markers)
            cell_type_to_marker[cell_type].update(markers)

    # Save to JSON files
    for category, cat_to_marker in [('tissue', tissue_to_marker), ('cell_type', cell_type_to_marker)]:
        json_file_path = os.path.join(output_directory, f"{'HPA' if is_hpa else 'hECA'}_{category}_to_marker.json")
        with open(json_file_path, 'w') as json_file:
            json.dump({k: list(v) for k, v in cat_to_marker.items()}, json_file, indent=4)
        print(f"{'HPA' if is_hpa else 'hECA'} {category} to marker JSON saved at: {json_file_path}")

# Correct file paths before running the function
hpa_excel_file_path = 'ontology/HPA_marker_reference.xlsx'
heca_excel_file_path = 'ontology/uHAF_marker_reference.xlsx'
output_directory = 'ontology/json_output'

# Make sure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Convert and save the HPA Excel file to JSON
convert_to_json(hpa_excel_file_path, output_directory, is_hpa=True)

# Convert and save the hECA Excel file to JSON
convert_to_json(heca_excel_file_path, output_directory, is_hpa=False)


HPA tissue to marker JSON saved at: ontology/json_output/HPA_tissue_to_marker.json
HPA cell_type to marker JSON saved at: ontology/json_output/HPA_cell_type_to_marker.json
hECA tissue to marker JSON saved at: ontology/json_output/hECA_tissue_to_marker.json
hECA cell_type to marker JSON saved at: ontology/json_output/hECA_cell_type_to_marker.json


In [5]:
import json

def create_custom_mapping():
    """
    Create a custom mapping from plural forms (common in HPA) to singular forms (common in hECA).
    This helps in aligning cell types between the two datasets.
    """
    return {
        'b cells': 'b cell',
        'dendritic cells': 'dendritic cell',
        'endothelial cells': 'endothelial cell',
        'fibroblasts': 'fibroblast',
        'granulocytes': 'granulocyte',
        'macrophages': 'macrophage',
        'monocytes': 'monocyte',
        'smooth muscle cells': 'smooth muscle cell',
        't cells': 't cell',
        'enterocytes': 'enterocyte',
        'goblet cells': 'goblet cell',
        'paneth cells': 'paneth cell',
        'enterocytes': 'enterocyte',
        'cone photoreceptor cells': 'cone cell',
        'horizontal cells': 'horizontal cell',
        'muller glia cells': 'muller cell',
        'rod photoreceptor cells': 'rod cell',
        'cardiomyocytes': 'cardiomyocyte cell',
        'collecting duct cells': 'collecting duct',
        'distal tubular cells': 'distal convoluted tubule',
        'proximal tubular cells': 'proximal convoluted tubule',
        'erythroid cells': 'erythroid cell',
        'hepatocytes': 'hepatocyte',
        'kupffer cells': 'kupffer cell',
        'bronchial epithelium, club cells': 'club cell/bronchiolar exocrine cell/clara cell',
        'alveolar cells type 1': 'type I alveolar cell/type I pneumocyte',
        'alveolar cells type 2': 'type II alveolar cell/type II pneumocyte',
        'exocrine glandular cells': 'exocrine cell',
        'ductal cells': 'ductal cell',
        'pancreatic endocrine cells': 'endocrine cell',
        'exocrine glandular cells': 'exocrine cell',
        'basal glandular cells': 'basal cell',
        'suprabasal keratinocytes': 'keratinocyte',
        'melanocytes': 'melanocyte',
        'paneth cells ': 'paneth cell',
        'sertoli cells': 'sertoli cell',
        'spermatogonia ': 'differentiating spermatogonia',
        'spermatogonia ': 'differentiated spermatogonia',
        # Add more mappings as needed based on the cell types in your datasets
    }

def normalize_name(name, custom_mapping):
    """
    Normalize cell type names by applying custom mappings for known discrepancies.
    """
    normalized = name.strip().lower().replace("-", " ").replace("_", " ")
    # Convert plural to singular by checking custom mapping
    normalized = custom_mapping.get(normalized, normalized)
    return normalized

def load_and_normalize_data(file_path, custom_mapping):
    """
    Load JSON data from a file, normalize cell type names using the custom mapping,
    and return a dictionary with normalized cell type names as keys.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    # Normalize cell type names and return a new dictionary
    return {normalize_name(key, custom_mapping): set(value) for key, value in data.items()}

# Define your custom mapping
custom_mapping = create_custom_mapping()

# Load and normalize the data from JSON files
heca_cell_type_to_marker = load_and_normalize_data('ontology/json_output/hECA_cell_type_to_marker.json', custom_mapping)
hpa_cell_type_to_marker = load_and_normalize_data('ontology/json_output/HPA_cell_type_to_marker.json', custom_mapping)

common_cell_types = set(heca_cell_type_to_marker.keys()) & set(hpa_cell_type_to_marker.keys())
print(common_cell_types)
print('\n')

similarity_stats = {}

for cell_type in common_cell_types:
    heca_markers = heca_cell_type_to_marker[cell_type]
    hpa_markers = hpa_cell_type_to_marker[cell_type]
    common_markers = heca_markers & hpa_markers
    
    total_markers_heca = len(heca_markers)
    total_markers_hpa = len(hpa_markers)
    total_common_markers = len(common_markers)
    
    similarity_stats[cell_type] = {
        "common_markers": total_common_markers,
        "percentage_common_markers_heca": round((total_common_markers / total_markers_heca) * 100, 1) if total_markers_heca else 0,
        "percentage_common_markers_hpa": round((total_common_markers / total_markers_hpa) * 100, 1) if total_markers_hpa else 0,
    }

# Display the similarity statistics
for cell_type, stats in similarity_stats.items():
    print(f"{cell_type}: {stats}")


{'basal cell', 'keratinocyte', 'b cell', 'exocrine cell', 'melanocyte', 'proximal convoluted tubule', 'sertoli cell', 'rod cell', 'horizontal cell', 'ductal cell', 'paneth cell', 't cell', 'granulocyte', 'muller cell', 'goblet cell', 'endothelial cell', 'enterocyte', 'endocrine cell', 'dendritic cell', 'distal convoluted tubule', 'fibroblast', 'erythroid cell', 'macrophage', 'smooth muscle cell', 'club cell/bronchiolar exocrine cell/clara cell', 'cardiomyocyte cell', 'cone cell', 'monocyte', 'collecting duct', 'hepatocyte'}


basal cell: {'common_markers': 0, 'percentage_common_markers_heca': 0.0, 'percentage_common_markers_hpa': 0.0}
keratinocyte: {'common_markers': 3, 'percentage_common_markers_heca': 75.0, 'percentage_common_markers_hpa': 100.0}
b cell: {'common_markers': 2, 'percentage_common_markers_heca': 25.0, 'percentage_common_markers_hpa': 66.7}
exocrine cell: {'common_markers': 0, 'percentage_common_markers_heca': 0.0, 'percentage_common_markers_hpa': 0.0}
melanocyte: {'comm

In [5]:
import re
import json

def create_custom_mapping():
    """
    Define a mapping for cell type synonyms and specific exceptions.
    This function now focuses on special cases not handled by automatic pluralization and normalization.
    """
    return {
        # Explicit mappings for cases with missing words or different word ending
        'bronchial epithelium, club cells': 'club cell/bronchiolar exocrine cell/clara cell',
        'alveolar cells type 1': 'type I alveolar cell/type I pneumocyte',
        'alveolar cells type 2': 'type II alveolar cell/type II pneumocyte',
        'cone photoreceptor cells': 'cone cell',
        'muller glia cells': 'muller cell',
        'rod photoreceptor cells': 'rod cell',
        'cardiomyocytes': 'cardiomyocyte cell',
        'collecting duct cells': 'collecting duct',
        'distal tubular cells': 'distal convoluted tubule',
        'proximal tubular cells': 'proximal convoluted tubule',
        'exocrine glandular cells': 'exocrine cell',
        'basal glandular cells': 'basal cell',
        'suprabasal keratinocytes': 'keratinocyte',
        'spermatogonia ': 'differentiating spermatogonia',
        'spermatogonia ': 'differentiated spermatogonia',
        # Add or adjust mappings as needed for specific cases
    }

def normalize_name(name, custom_mapping):
    """
    Normalize cell type names by applying custom mappings for known discrepancies, 
    including automatic adjustments for case, hyphens, underscores, 
    and handling singular/plural forms relevant to cell types.
    """
    # Convert to lowercase and replace hyphens/underscores with spaces
    normalized = name.strip().lower().replace("-", " ").replace("_", " ")

    # Apply custom mappings first for specific synonyms and exceptions
    normalized = custom_mapping.get(normalized, normalized)

    # Automatically handle plural forms with basic English rules, tailored for cell types
    if normalized.endswith('ies'):
        normalized = re.sub('ies$', 'y', normalized)  # Correct rule for converting plurals ending in 'ies' to 'y'
    elif normalized.endswith('es'):
        # Correct handling for plurals ending in 'es', which might be common for certain biological terms
        normalized = normalized[:-2]  # Removes the 'es', e.g., "paneth cells" to "paneth cell"
    elif normalized.endswith('s') and not normalized.endswith('ss'):
        normalized = normalized[:-1]  # General case for plurals not ending in 'ss', e.g., "cells" to "cell"

    return normalized

def load_and_normalize_data(file_path, custom_mapping):
    """
    Load JSON data from a file, normalize cell type names using the custom mapping,
    and return a dictionary with normalized cell type names as keys.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    # Normalize cell type names and return a new dictionary
    return {normalize_name(key, custom_mapping): set(value) for key, value in data.items()}

# Define your custom mapping
custom_mapping = create_custom_mapping()

# Load and normalize the data from JSON files
heca_cell_type_to_marker = load_and_normalize_data('ontology/json_output/hECA_cell_type_to_marker.json', custom_mapping)
hpa_cell_type_to_marker = load_and_normalize_data('ontology/json_output/HPA_cell_type_to_marker.json', custom_mapping)

common_cell_types = set(heca_cell_type_to_marker.keys()) & set(hpa_cell_type_to_marker.keys())
print(common_cell_types)
print('\n')

similarity_stats = {}

for cell_type in common_cell_types:
    heca_markers = heca_cell_type_to_marker[cell_type]
    hpa_markers = hpa_cell_type_to_marker[cell_type]
    common_markers = heca_markers & hpa_markers
    
    total_markers_heca = len(heca_markers)
    total_markers_hpa = len(hpa_markers)
    total_common_markers = len(common_markers)
    
    # Only add to similarity_stats if total_common_markers is greater than 0
    if total_common_markers > 0:
        similarity_stats[cell_type] = {
            "common_markers": total_common_markers,
            "percentage_common_markers_heca": round((total_common_markers / total_markers_heca) * 100, 1) if total_markers_heca else 0,
            "percentage_common_markers_hpa": round((total_common_markers / total_markers_hpa) * 100, 1) if total_markers_hpa else 0,
        }

for cell_type, stats in similarity_stats.items():
    print(f"{cell_type}: {stats}")



{'rod cell', 'club cell/bronchiolar exocrine cell/clara cell', 'keratinocyte', 'collecting duct', 'horizontal cell', 'goblet cell', 'fibroblast', 'exocrine cell', 'urothelial cell', 'proximal convoluted tubule', 'muller cell', 'paneth cell', 'ductal cell', 'sertoli cell', 'erythroid cell', 't cell', 'smooth muscle cell', 'basal cell', 'b cell', 'cardiomyocyte cell', 'dendritic cell', 'distal convoluted tubule', 'endothelial cell', 'cone cell'}


rod cell: {'common_markers': 2, 'percentage_common_markers_heca': 50.0, 'percentage_common_markers_hpa': 66.7}
club cell/bronchiolar exocrine cell/clara cell: {'common_markers': 2, 'percentage_common_markers_heca': 22.2, 'percentage_common_markers_hpa': 66.7}
keratinocyte: {'common_markers': 3, 'percentage_common_markers_heca': 75.0, 'percentage_common_markers_hpa': 100.0}
horizontal cell: {'common_markers': 3, 'percentage_common_markers_heca': 100.0, 'percentage_common_markers_hpa': 100.0}
goblet cell: {'common_markers': 2, 'percentage_common_

In [7]:
# Import required libraries
%pip install pandas
import pandas as pd
import json
import os
from collections import defaultdict

class GeneExpressionAtlas:
    def __init__(self, data_path, columns_to_check):
        self.data_path = data_path
        self.columns_to_check = columns_to_check
        self.cell_types_to_ensembl = defaultdict(set)

    def extract_cell_types(self):
        df = pd.read_csv(self.data_path, sep='\t')
        for index, row in df.iterrows():
            ensembl_id = row['Ensembl']
            for col in self.columns_to_check:
                cell_type_data = row[col]
                if pd.notna(cell_type_data):
                    for item in cell_type_data.split(';'):
                        cell_type = item.split(':')[0].strip()
                        self.cell_types_to_ensembl[cell_type].add(ensembl_id)
        return self.cell_types_to_ensembl

    def to_json(self, output_path):
        # Convert sets to lists for JSON serialization
        for cell_type, ensembl_ids in self.cell_types_to_ensembl.items():
            self.cell_types_to_ensembl[cell_type] = list(ensembl_ids)
        with open(output_path, 'w') as f:
            json.dump(self.cell_types_to_ensembl, f)
        return output_path

class HPA(GeneExpressionAtlas):
    def __init__(self, data_path):
        columns_to_check = [
            "RNA tissue specific nTPM",
            "RNA single cell type specific nTPM",
            "RNA blood cell specific nTPM",
            "RNA blood lineage specific nTPM"
        ]
        super().__init__(data_path, columns_to_check)

class hECA(GeneExpressionAtlas):
    # hECA specific implementation would go here, if needed.
    pass

# Example usage for HPA
hpa_data_path = "unzipped_folder/proteinatlas.tsv"
hpa = HPA(hpa_data_path)

# Extract cell types and associated Ensembl Gene IDs
hpa_cell_types_to_ensembl = hpa.extract_cell_types()

# Directory where the aggregated results will be saved
output_directory = "output_files"

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Write the results to a JSON file
json_file_path = hpa.to_json(f'{output_directory}/cell_types_to_ensembl.json')
print(f"Data written to {json_file_path}")


Note: you may need to restart the kernel to use updated packages.
Data written to output_files/cell_types_to_ensembl.json


In [2]:
''''
%pip install pandas
import pandas as pd

class GeneExpressionAtlas:
    def __init__(self, data_path, columns_to_check, custom_mapping=None):
        self.data_path = data_path
        self.columns_to_check = columns_to_check
        self.custom_mapping = custom_mapping if custom_mapping else create_custom_mapping()
        self.cell_types_to_ensembl = defaultdict(set)

    def extract_cell_types(self):
        df = pd.read_csv(self.data_path, sep='\t')
        for _, row in df.iterrows():
            ensembl_id = row['Ensembl']
            for col in self.columns_to_check:
                cell_type_data = row[col]
                if pd.notna(cell_type_data):
                    for item in cell_type_data.split(';'):
                        cell_type, _ = item.split(':')  # Assume format is "cell_type:expression"
                        normalized_cell_type = normalize_name(cell_type, self.custom_mapping)
                        self.cell_types_to_ensembl[normalized_cell_type].add(ensembl_id)
        return self.cell_types_to_ensembl

intersection = {}
for cell_type in hpa_cell_types_to_ensembl:
    if cell_type in heca_cell_type_to_marker:
        intersection[cell_type] = {
            'HPA_Ensembl_IDs': list(hpa_cell_types_to_ensembl[cell_type]),
            'hECA_markers': heca_cell_type_to_marker[cell_type]
        }

# Print the intersection
for cell_type, data in intersection.items():
    print(f"Cell Type: {cell_type}")
    print(f"HPA Ensembl Gene IDs: {data['HPA_Ensembl_IDs']}")
    print(f"hECA Marker Genes: {data['hECA_markers']}\n")
'''

'\'\n%pip install pandas\nimport pandas as pd\n\nclass GeneExpressionAtlas:\n    def __init__(self, data_path, columns_to_check, custom_mapping=None):\n        self.data_path = data_path\n        self.columns_to_check = columns_to_check\n        self.custom_mapping = custom_mapping if custom_mapping else create_custom_mapping()\n        self.cell_types_to_ensembl = defaultdict(set)\n\n    def extract_cell_types(self):\n        df = pd.read_csv(self.data_path, sep=\'\t\')\n        for _, row in df.iterrows():\n            ensembl_id = row[\'Ensembl\']\n            for col in self.columns_to_check:\n                cell_type_data = row[col]\n                if pd.notna(cell_type_data):\n                    for item in cell_type_data.split(\';\'):\n                        cell_type, _ = item.split(\':\')  # Assume format is "cell_type:expression"\n                        normalized_cell_type = normalize_name(cell_type, self.custom_mapping)\n                        self.cell_types_to_ensem

In [9]:
import requests
import gzip
import os

class GeneExpressionAtlas:
    def __init__(self, data_path, columns_to_check, custom_mapping=None):
        self.data_path = data_path
        self.columns_to_check = columns_to_check
        self.custom_mapping = custom_mapping if custom_mapping else self.create_custom_mapping()
        self.cell_types_to_ensembl = defaultdict(set)
        
        # Download required files before loading them
        self.download_and_prepare_ontology_files()

        # Now it's safe to load the GO DAG and gene2go data
        if self.go_obo_path:
            self.go_dag = GODag(self.go_obo_path)
        if self.gene2go_path:
            self.gene2go = read_ncbi_gene2go(self.gene2go_path, taxids=[9606])  # Assuming human genes

    def create_custom_mapping(self):
        # Placeholder for your custom mapping function
        return {}

    def download_and_prepare_ontology_files(self):
        # Specify the ontology folder and file URLs
        ontology_folder = "ontology"
        self.go_obo_path = os.path.join(ontology_folder, "go-basic.obo")
        self.gene2go_path = os.path.join(ontology_folder, "gene2go")
        
        go_obo_url = "http://purl.obolibrary.org/obo/go/go-basic.obo"
        gene2go_url = "ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz"

        # Download the files
        self.download_file(go_obo_url, ontology_folder)
        self.download_file(gene2go_url, ontology_folder, decompress=True)

    def download_file(self, url, dest_folder, decompress=False):
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)  # Create directory if it does not exist

        filename = url.split('/')[-1]
        file_path = os.path.join(dest_folder, filename)
        
        # Download the file
        print(f"Downloading {filename}...")
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            if decompress and filename.endswith('.gz'):
                with gzip.open(file_path, 'rb') as f_in:
                    with open(file_path[:-3], 'wb') as f_out:  # Remove .gz extension for the output file
                        f_out.write(f_in.read())
                os.remove(file_path)  # Remove the compressed file
                print(f"Decompressed {filename}")
        else:
            print(f"Error downloading {filename}: Status Code {response.status_code}")

# Further implementation of GeneExpressionAtlas methods...

# Example usage:
gene_expression_atlas = GeneExpressionAtlas(
    data_path="path/to/proteinatlas.tsv",
    columns_to_check=["column1", "column2"]
)



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


ValueError: COULD NOT READ(ontology/go-basic.obo)
download obo file first
 [http://geneontology.org/ontology/go-basic.obo]

In [8]:
# Import necessary libraries
from collections import defaultdict
import json

# Function to load a genelist from a file
def load_genelist(file_path):
    with open(file_path, 'r') as f:
        # Remove quotes and strip whitespace from each line
        return [line.strip().strip('"') for line in f.read().splitlines()]

# Load the cell_types_to_ensembl.json file
cell_types_to_ensembl_filepath = 'output_files/cell_types_to_ensembl.json'
with open(cell_types_to_ensembl_filepath, 'r') as f:
    cell_types_to_ensembl = json.load(f)

# Load all genelists using file paths for the gene lists
genelists_files = [f'genelists/Genelist{i}.txt' for i in range(1, 7)]

# Initialize a dictionary to store cell type frequencies for all genelists
cell_types_for_all_genelists = {}

# Loop through each genelist file
for i, genelist_file in enumerate(genelists_files, 1):
    # Load the current genelist
    genelist = load_genelist(genelist_file)
    
    # Initialize a defaultdict to store the results for the current genelist
    cell_types_for_genelist = defaultdict(int)

    # Identify cell types associated with the genes in the current genelist
    for cell_type, ensembl_ids in cell_types_to_ensembl.items():
        for ensembl_id in genelist:
            if ensembl_id in ensembl_ids:
                cell_types_for_genelist[cell_type] += 1

    # Store the results for the current genelist
    cell_types_for_all_genelists[f'Genelist{i}'] = cell_types_for_genelist

# Save the aggregated results to a JSON file
cell_types_for_all_genelists_file = 'output_files/cell_types_for_all_genelists.json'
with open(cell_types_for_all_genelists_file, 'w') as f:
    json.dump(cell_types_for_all_genelists, f)

cell_types_for_all_genelists_file

'output_files/cell_types_for_all_genelists.json'

In [9]:
import pandas as pd

# Load the protein atlas data
protein_atlas_filepath = 'unzipped_folder/proteinatlas.tsv'
protein_atlas_df = pd.read_csv(protein_atlas_filepath, sep='\t')

# Extract the Ensembl IDs from the 'Gene' column
ensembl_ids = protein_atlas_df['Ensembl'].unique().tolist()

# Save the Ensembl IDs to a JSON file
ensembl_ids_filepath = 'output_files/protein_atlas_ensembl_ids.json'
with open(ensembl_ids_filepath, 'w') as f:
    json.dump(ensembl_ids, f)

# Check the first 5 Ensembl IDs
ensembl_ids[:5], ensembl_ids_filepath

(['ENSG00000000003',
  'ENSG00000000005',
  'ENSG00000000419',
  'ENSG00000000457',
  'ENSG00000000460'],
 'output_files/protein_atlas_ensembl_ids.json')

In [10]:
%pip install scipy statsmodels
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
from collections import defaultdict
import json

# Load all genelists using file paths for the gene lists
genelist_filepaths = {f'Genelist{i}': f'genelists/Genelist{i}.txt' for i in range(1, 7)}

# Paths to the input files
cell_types_to_ensembl_filepath = 'output_files/cell_types_to_ensembl.json'
protein_atlas_ensembl_ids_filepath = 'output_files/protein_atlas_ensembl_ids.json'

# Load the Protein Atlas Ensembl IDs
with open(protein_atlas_ensembl_ids_filepath, 'r') as f:
    protein_atlas_ensembl_ids = set(json.load(f))

# Load the mapping of cell types to Ensembl IDs
with open(cell_types_to_ensembl_filepath, 'r') as f:
    cell_types_to_ensembl = json.load(f)

# Initialize a dictionary to store the results
fisher_test_results = defaultdict(dict)

# Perform Fisher's Exact Test for each genelist
for genelist_name, genelist_filepath in genelist_filepaths.items():
    # Load the genelist
    with open(genelist_filepath, 'r') as f:
        genelist = set(line.strip().strip('"') for line in f.readlines())

    # Total number of genes in the genelist and in the Protein Atlas
    total_genes_genelist = len(genelist)
    total_genes_atlas = len(protein_atlas_ensembl_ids)

    p_values = []

    # Perform the test for each cell type
    for cell_type, ensembl_ids in cell_types_to_ensembl.items():
        ensembl_ids_set = set(ensembl_ids)

        # Count of genes in both the genelist and the cell type
        count_in_both = len(genelist.intersection(ensembl_ids_set))
        count_in_genelist_not_cell_type = len(genelist.difference(ensembl_ids_set))
        count_in_cell_type_not_genelist = len(ensembl_ids_set.difference(genelist))
        count_in_neither = total_genes_atlas - (count_in_both + count_in_genelist_not_cell_type + count_in_cell_type_not_genelist)

        # Construct the contingency table
        table = [
            [count_in_both, count_in_cell_type_not_genelist],
            [count_in_genelist_not_cell_type, count_in_neither]
        ]

        # Perform Fisher's Exact Test
        odds_ratio, p_value = fisher_exact(table, alternative='greater')
        p_values.append(p_value)

        # Store the results without adjusted p-values first
        fisher_test_results[genelist_name][cell_type] = {
            'p_value': p_value,
            'odds_ratio': odds_ratio,
            'count_in_both': count_in_both,
            'count_in_genelist_not_cell_type': count_in_genelist_not_cell_type,
            'count_in_cell_type_not_genelist': count_in_cell_type_not_genelist,
            'count_in_neither': count_in_neither
        }

    # Adjust p-values using the Benjamini-Hochberg procedure
    _, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

    # Store the adjusted p-values in the results
    for (cell_type, _), adj_p_value in zip(fisher_test_results[genelist_name].items(), pvals_corrected):
        fisher_test_results[genelist_name][cell_type]['adjusted_p_value'] = adj_p_value
    
    # Sort results by adjusted p_value in ascending order
    fisher_test_results[genelist_name] = dict(sorted(fisher_test_results[genelist_name].items(), key=lambda x: x[1].get('adjusted_p_value', 1)))

# Save the results to a JSON file
results_filepath = 'output_files/fisher_test_results.json'
with open(results_filepath, 'w') as f:
    json.dump(fisher_test_results, f)

results_filepath


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.8/37.8 MB 51.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 101.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 51.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


'output_files/fisher_test_results.json'

In [11]:
%pip install statsmodels
import json
from statsmodels.stats.multitest import multipletests

# Load Fisher test results
results_filepath = 'output_files/fisher_test_results.json'
with open(results_filepath, 'r') as file:
    fisher_test_results = json.load(file)

# Extract insights from the Fisher test results
insights = {}
p_value_threshold = 0.05  # Adjusted p-value threshold

for genelist, cell_types in fisher_test_results.items():
    associations = []
    p_values = []
    for cell_type, stats in cell_types.items():
        p_value = stats.get('p_value', 1)
        count_in_both = stats.get('count_in_both', 0)
        count_in_genelist_not_cell_type = stats.get('count_in_genelist_not_cell_type', 0)
        count_in_cell_type_not_genelist = stats.get('count_in_cell_type_not_genelist', 0)
        count_in_neither = stats.get('count_in_neither', 0)

        # Calculate the odds ratio
        odds_ratio = (count_in_both * count_in_neither) / (count_in_genelist_not_cell_type * count_in_cell_type_not_genelist) if count_in_genelist_not_cell_type and count_in_cell_type_not_genelist else float('inf')

        associations.append({
            'cell_type': cell_type,
            'p_value': p_value,
            'odds_ratio': odds_ratio,  # Include the odds ratio
            'count_in_both': count_in_both,
            'count_in_genelist_not_cell_type': count_in_genelist_not_cell_type,
            'count_in_cell_type_not_genelist': count_in_cell_type_not_genelist,
            'count_in_neither': count_in_neither
        })
        p_values.append(p_value)

    # Adjust p-values using the Benjamini-Hochberg procedure
    reject, pvals_corrected, _, _ = multipletests(p_values, alpha=p_value_threshold, method='fdr_bh')

    # Store only associations with an adjusted p-value below the threshold and less than 1.0
    significant_associations = []
    for association, adj_p_value, rej in zip(associations, pvals_corrected, reject):
        if rej and adj_p_value < 1.0:
            association['adjusted_p_value'] = adj_p_value
            significant_associations.append(association)

    # Sort the significant associations by adjusted p_value in ascending order
    significant_associations.sort(key=lambda x: x.get('adjusted_p_value', 1))
    insights[genelist] = significant_associations

# Write the insights to a JSON file
output_filepath = 'output_files/fisher_test_insights.json'
with open(output_filepath, 'w') as f:
    json.dump(insights, f)

# Displaying the first few insights for review
{key: insights[key][:5] for key in insights.keys()}


Note: you may need to restart the kernel to use updated packages.


{'Genelist1': [{'cell_type': 'esophagus',
   'p_value': 7.601491563693231e-83,
   'odds_ratio': 38.03038138332256,
   'count_in_both': 81,
   'count_in_genelist_not_cell_type': 119,
   'count_in_cell_type_not_genelist': 351,
   'count_in_neither': 19611,
   'adjusted_p_value': 1.0262013610985863e-80},
  {'cell_type': 'Suprabasal keratinocytes',
   'p_value': 4.524016635921081e-70,
   'odds_ratio': 26.866237987563593,
   'count_in_both': 78,
   'count_in_genelist_not_cell_type': 122,
   'count_in_cell_type_not_genelist': 464,
   'count_in_neither': 19498,
   'adjusted_p_value': 3.0537112292467296e-68},
  {'cell_type': 'vagina',
   'p_value': 1.6908530223420254e-62,
   'odds_ratio': 65.7748344370861,
   'count_in_both': 49,
   'count_in_genelist_not_cell_type': 151,
   'count_in_cell_type_not_genelist': 98,
   'count_in_neither': 19864,
   'adjusted_p_value': 7.608838600539114e-61},
  {'cell_type': 'Basal keratinocytes',
   'p_value': 7.369658263665469e-38,
   'odds_ratio': 21.5081838252